<a href="https://colab.research.google.com/github/SamLutz/2024_MLEES/blob/main/final_project_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Final Project MLEES SA24


In [3]:
!pip install netCDF4
!pip install xarray
import os
import numpy as np
import xarray as xr
from netCDF4 import Dataset
from datetime import datetime
import pooch
import netCDF4 as nc
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.9 MB/s eta 0:00:00


Download data from Onedrive with Pooch


In [4]:
# Liste des fichiers OneDrive avec leurs URL
files_urls = [
    'https://unils-my.sharepoint.com/:u:/g/personal/samuel_lutz_unil_ch/Efe70_k3wXJHnrbYK9h1uBwBUgNCzc5_24r73e1xETG0ug?download=1',
    'https://unils-my.sharepoint.com/:u:/g/personal/samuel_lutz_unil_ch/EWAfM2fXmJBEiCTivTpZlCEBa5sKr-6SGX8M27JoblF3Tg?download=1',
    'https://unils-my.sharepoint.com/:u:/g/personal/samuel_lutz_unil_ch/ESbZRJJ1jSlCuRxFQTeXxmEBJ8yD8u42W8cjlNVbeh4CCw?download=1',
    'https://unils-my.sharepoint.com/:u:/g/personal/samuel_lutz_unil_ch/ERypBJ2Dpi1LgIYALQoJFhoBHpjHhC5Ygk85FmJ075y7uw?download=1',
]

# Hashed values des fichiers (facultatif, utiliser None si inconnu)
hashes = [None] * len(files_urls)  # None pour tous les fichiers si tu ne connais pas leurs empreintes

In [5]:
downloaded_files = []  # Liste pour stocker les chemins des fichiers téléchargés

for url, file_hash in zip(files_urls, hashes):
    # Télécharger chaque fichier
    file_path = pooch.retrieve(url, known_hash=file_hash)
    downloaded_files.append(file_path)

print("Fichiers téléchargés :", downloaded_files)

SHA256 hash of downloaded file: ed7070a75e387b8a58b6aaea243f0bb48c9c7722c65c694c11f62bad24c2ebc1
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: 11b06a7ded09e4b6bd492b4691364871c1457610f096eb654c41711dcf4ea001
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: 03b7d4402715cff1a26b748d0e4df5338ef28dd7e0a3c20ab758155d8bcc234a
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: 5e35b275b7b3879558b5e604760e8a904e01f2fe83ee3ee1f6d977aba6a47ac1
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.


Fichiers téléchargés : ['/root/.cache/pooch/eeebeae5c5966a8643cbec3a97f28bca-Efe70_k3wXJHnrbYK9h1uBwBUgNCzc5_24r73e1xETG0ug', '/root/.cache/pooch/15e08fe31f9592ff42cc7caecb0b9267-EWAfM2fXmJBEiCTivTpZlCEBa5sKr-6SGX8M27JoblF3Tg', '/root/.cache/pooch/34af5befd86ba7cc499434f6c106d34a-ESbZRJJ1jSlCuRxFQTeXxmEBJ8yD8u42W8cjlNVbeh4CCw', '/root/.cache/pooch/364252904277340c3a219805eacf226a-ERypBJ2Dpi1LgIYALQoJFhoBHpjHhC5Ygk85FmJ075y7uw']


Create a liste with all variables in data

In [6]:
# Liste pour stocker les variables de tous les fichiers
all_variables = []

for file_path in downloaded_files:
    with nc.Dataset(file_path) as dataset:
        print(f"Fichier : {file_path}")
        print("Variables disponibles :", dataset.variables.keys())
        all_variables.append(dataset.variables.keys())


Fichier : /root/.cache/pooch/eeebeae5c5966a8643cbec3a97f28bca-Efe70_k3wXJHnrbYK9h1uBwBUgNCzc5_24r73e1xETG0ug
Variables disponibles : dict_keys(['Batt', 'Ptemp', 'AirTC', 'RH', 'Slrw', 'Slrm', 'WS', 'WindDir', 'Rain', 'BP', 'WindGust', 'time_qual', 'Batt_qual', 'Ptemp_qual', 'AirTC_qual', 'RH_qual', 'Slrw_qual', 'Slrm_qual', 'WS_qual', 'WindDir_qual', 'Rain_qual', 'BP_qual', 'WindGust_qual', 'time'])
Fichier : /root/.cache/pooch/15e08fe31f9592ff42cc7caecb0b9267-EWAfM2fXmJBEiCTivTpZlCEBa5sKr-6SGX8M27JoblF3Tg
Variables disponibles : dict_keys(['temp', 'surfacetemp', 'bottomtemp', 'mixed_layer_depth', 'thermocline_depth', 'schmidt_stability', 'heat_content', 'time', 'depth'])
Fichier : /root/.cache/pooch/34af5befd86ba7cc499434f6c106d34a-ESbZRJJ1jSlCuRxFQTeXxmEBJ8yD8u42W8cjlNVbeh4CCw
Variables disponibles : dict_keys(['temp', 'interp_temp', 'surfacetemp', 'bottomtemp', 'mixed_layer_depth', 'thermocline_depth', 'schmidt_stability', 'heat_content', 'time', 'depth'])
Fichier : /root/.cache/poo

In [8]:
# Charger tous les fichiers en tant que Dataset
datasets = [xr.open_dataset(file) for file in downloaded_files]

# Fusionner sur une dimension commune (par exemple "time")
data = xr.concat(datasets, dim="time")

# Inspecter le résultat
print(data)

# Sauvegarder le résultat fusionné
data.to_netcdf("data_merged.nc")


<xarray.Dataset> Size: 562MB
Dimensions:            (time: 516443, depth: 53)
Coordinates:
  * time               (time) datetime64[ns] 4MB 2019-07-29T18:00:00 ... 2024...
  * depth              (depth) float64 424B 0.25 0.5 0.75 1.0 ... 87.0 90.0 93.0
Data variables: (12/31)
    Batt               (time) float64 4MB 12.24 12.24 12.24 ... nan nan nan
    Ptemp              (time) float64 4MB 30.39 29.73 29.1 28.74 ... nan nan nan
    AirTC              (time) float64 4MB 23.39 23.31 23.43 ... nan nan nan
    RH                 (time) float64 4MB 60.12 60.19 58.28 57.7 ... nan nan nan
    Slrw               (time) float64 4MB 61.07 62.85 113.0 95.0 ... nan nan nan
    Slrm               (time) float64 4MB 0.03664 0.03771 0.0678 ... nan nan nan
    ...                 ...
    bottomtemp         (time) float64 4MB nan nan nan nan ... 7.499 7.507 7.507
    mixed_layer_depth  (time) float64 4MB nan nan nan nan ... 19.0 15.0 15.0
    thermocline_depth  (time) float64 4MB nan nan nan nan ... 

In [11]:
# Boucle sur les fichiers NetCDF téléchargés
for file_path in downloaded_files:
    # Ouvrir le fichier NetCDF
    with Dataset(file_path, 'r') as nc_file:
        # Extraire les variables
        unix_time = nc_file.variables['time'][:]
        air_temp = nc_file.variables['AirTC'][:]
        solar_radiation = nc_file.variables['Slrw'][:]
        wind_speed = nc_file.variables['WS'][:]

        # Convertir le temps Unix en datetime
        datetime_times = [datetime.utcfromtimestamp(t) for t in unix_time]

        # Créer un tableau structuré avec les données
        file_data = np.array(
            list(zip(datetime_times, air_temp, solar_radiation, wind_speed)),
            dtype=[('time', 'O'), ('AirTC', 'f4'), ('SlrW', 'f4'), ('WS', 'f4')]
        )
        # Ajouter les données extraites à la liste
        all_variables.append(file_data)

# Combiner toutes les données dans un seul tableau NumPy
meteo = np.concatenate(all_variables)

# Vérifier les contenus
print(meteo[:5])  # Afficher les 5 premiers enregistrements


<ipython-input-11-b19ab0e74c47>:15: UserWarning: Warning: converting a masked element to nan.
  file_data = np.array(


KeyError: 'AirTC'

Random forest model

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Charger les données (partant du tableau `meteo`)
# Ajouter une variable fictive "water_temp" pour la démonstration
np.random.seed(42)  # Pour des résultats reproductibles
meteo = np.array(
    list(zip(meteo['AirTC'], meteo['SlrW'], meteo['WS'], np.random.uniform(10, 20, len(meteo)))),
    dtype=[('AirTC', 'f4'), ('SlrW', 'f4'), ('WS', 'f4'), ('water_temp', 'f4')]
)


In [ ]:
# Variables explicatives (features) et variable cible
X = np.vstack((meteo['AirTC'], meteo['SlrW'], meteo['WS'])).T
y = meteo['water_temp']

In [ ]:
# Diviser les données en ensembles 3/5 entraînement, 1/5 validation, 1/5 test
n_samples = len(X)
train_size = int(n_samples * 3 / 5)
val_size = int(n_samples * 1 / 5)

X_train, X_val, X_test = X[:train_size], X[train_size:train_size+val_size], X[train_size+val_size:]
y_train, y_val, y_test = y[:train_size], y[train_size:train_size+val_size], y[train_size+val_size:]

# Vérifier les tailles des ensembles
print(f"Taille des ensembles :")
print(f"Entraînement : {len(X_train)}, Validation : {len(X_val)}, Test : {len(X_test)}")

In [ ]:
# Créer et entraîner le modèle Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Évaluer le modèle sur l'ensemble de validation
y_val_pred = rf_model.predict(X_val)
val_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
val_r2 = r2_score(y_val, y_val_pred)
print(f"Validation - RMSE : {val_rmse:.3f}, R² : {val_r2:.3f}")

In [ ]:
# Tester le modèle sur l'ensemble de test
y_test_pred = rf_model.predict(X_test)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
test_r2 = r2_score(y_test, y_test_pred)
print(f"Test - RMSE : {test_rmse:.3f}, R² : {test_r2:.3f}")

# Visualiser les performances sur l'ensemble de test
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_test_pred, alpha=0.7, label="Prédictions vs Réel")
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--r', label="Idéal")
plt.xlabel("Valeurs réelles (°C)")
plt.ylabel("Valeurs prédites (°C)")
plt.title("Prédiction de la température de l'eau - Ensemble de test")
plt.legend()
plt.show()